# Spotify Data EDA

## data collection

* Use Spotipy to get Spotify public playlists data
* For each playlist, we get our response variable: # of followers
* For predictors, we pick the following attributes:
    * the number of tracks in the playlist
    * Artists info (for this part, we mainly focus on the popularity of artists in each track)
        * We calculated the mean popularity of all artists in the playlist
        * And the standard deviation, which might tell us some information about variaty of the playlists
    * Avaiable markets: if this playlist can reach wider range, it might get more followers

In [181]:
import pandas as pd 
import numpy as np
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

client_credentials_manager = SpotifyClientCredentials(client_id, client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

playlists = sp.user_playlists('spotify')
playlists_data = []
while playlists:
    for i, playlist in enumerate(playlists['items']):
        playlist_data = {}
        playlist_data["songs"] = []
        
        uri = playlist["uri"]
        username = uri.split(':')[2]
        playlist_id = uri.split(':')[4]
        
        results = sp.user_playlist(username, playlist_id)
        
        playlist_data["playlist_id"] = playlist_id

        playlist_data["followers"] = results['followers']['total']
        
        playlist_data["tracks_num"] = playlist['tracks']['total']
        
        playlist_data["name"] = playlist["name"]
        if i%100 == 0:
            print(i,playlist_data["name"])
        
        tracks = sp.user_playlist_tracks("spotify",playlist_id)
        markets = []
        pop = []
        for j in range(tracks_num):
            try:
                playlist_data["songs"].append(tracks["items"][j]["track"]['name']) 
                
                market = tracks["items"][i]["track"]["available_markets"]
                markets = list(set().union(markets,market))
                
                pop.append(tracks["items"][j]["track"]['popularity'])
                
            except:
                continue
        playlist_data["avaiable_market"] = len(markets)
        if pop != []:
            playlist_data["pop_mean"] = np.mean(pop)
            playlist_data["pop_std"] = np.std(pop)
        else:
            playlist_data["pop_mean"] = np.nan
            playlist_data["pop_std"] = np.nan

        playlists_data.append(playlist_data)
    
    if playlists['next']:
        playlists = sp.next(playlists)
    else:
        playlists = None

## Store data in json form and read

In [2]:
import json
# with open("playlists_data.json","w") as js:
#     json.dump(playlists_data,js)
# playlists_dataframe = pd.DataFrame.from_dict(playlists_data)
# playlists_dataframe.to_json("playlists_df.json")

In [3]:
# playlists_dataframe= pd.read_json("playlists_df.json")
with open('data.json','r') as js:
    pl=json.load(js)

In [4]:
def audio_features(self, tracks=[]):
    """ Get audio features for one or multiple tracks based upon their Spotify IDs
    Parameters:
    - tracks - a list of track URIs, URLs or IDs, maximum: 50 ids
    """
    if isinstance(tracks, str):
        trackid = self._get_id('track', tracks)
        results = self._get('audio-features/?ids=' + trackid)
    else:
        tlist = [self._get_id('track', t) for t in tracks]
        results = self._get('audio-features/?ids=' + ','.join(tlist))
    # the response has changed, look for the new style first, and if
    # its not there, fallback on the old style
    if 'audio_features' in results:
        return results['audio_features']
    else:
        return results

In [13]:
with open('data.json','r') as js:
    pl=json.load(js)

In [11]:
for playlist in pl:
    for track in playlist['tracks']:
        if track != None:
            name=track['artists']['name']

Index(['list', 'tracks'], dtype='object')